### (Demo) Connect 4

This document explains the implementation of a demo connect 4 environment, and players to play in that environment.

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from ipywidgets import widgets

%matplotlib inline

from game.widget import WidgetGame, game_widget_for, player_widget_for

In [2]:
game = WidgetGame()
game_widget_for(game)

In [3]:
game = WidgetGame()
widgets.HBox(children=[player_widget_for(game), game_widget_for(game)])